Theoretical part:
    
1. Hypothesis Testing

(a) P(mth experiment gives significant result | m experiments lacking power to reject H0)? <br>
The test's probability of making a type 1 error is $\alpha$ and increases when experiments are repeated.
The first m-1 experiments will have to return an insignificant result, which means they don't have the power to reject the null hypothesis. This would be $(1 - \alpha)$. The probability that the first m-1 experiments gives this result is thus $(1 - \alpha)^m$. The $m^{th}$ should be a type 1 error. The total probability is therefore $\alpha * (1 - \alpha)^m$

(b) P(at least one significant result | m experiments lacking power to reject H0)? <br>
<!--For the first test, there is $\alpha$ probability for a significant result, for the second it is $(1 - \alpha) * \alpha$. It continues like this until m tests are reached. --> <!-- sum of this? -->

The family-wise error rate can be used for the probability of making one or type 1 errors when performing multiple hypothesis tests. Many procedures to control the familywise error rate exist for this, one we could easily use is the Šidák procedure which tests the hypothesis using the formula $1 - (1 - \alpha)^\frac{1}{m}$. When the $p_i$ is lower or equal to this measure it rejects the null hypothesis.   

<!-- this doesn't really answer the question i think? -->

2. Bias and unfairness in Interleaving experiments

In a situation where we have two lists of documents: $l_1$ with documents $d_1$, $d_2$ and $d_3$ and $l_2$ with documents $d_2$, $d_3$ and $d_4$, both ranked in that order. When only document 3 is relevant, $l_2$ should win every time, since it has a higher ranking. However, with Team Draft interleaving, this does not happen, because the merging to the interleaved happens with coin flips and documents will be skipped when they have already been supplied by another list. When document 3 is the only relevant document, what happens in this situation is that after the skipping, both lists now have document 3 as their next document in the list, which means the coin flip doesn't matter and both lists will have the same relevance, as document 3 is the only relevant document, even though one of the lists is clearly better.

Experimental Part

Step 1

In [4]:
import math
import pprint
import itertools
import numpy as np

pp = pprint.PrettyPrinter(indent=4)

In [5]:
relevance = ['N', 'R', 'HR']

pairs = []

# All permutations of graded relevances of length 5
permutations = list(itertools.product(relevance, repeat=5))

# All possible pairs of P and E assuming they can give the same outcome
for perm in permutations:
    for perm_2 in permutations:
        pairs.append([perm, perm_2])
        
# Show 5000 random pairs uniformly selected
indices = np.random.randint(0, len(pairs), 5000)
sample_pairs = [pairs[i] for i in indices]
pp.pprint(sample_pairs)

[   [('HR', 'HR', 'N', 'N', 'R'), ('R', 'N', 'R', 'R', 'N')],
    [('R', 'R', 'R', 'HR', 'HR'), ('HR', 'R', 'R', 'R', 'R')],
    [('R', 'R', 'R', 'R', 'R'), ('R', 'R', 'HR', 'R', 'N')],
    [('HR', 'R', 'R', 'N', 'R'), ('HR', 'R', 'HR', 'HR', 'R')],
    [('HR', 'N', 'R', 'N', 'R'), ('HR', 'R', 'N', 'N', 'N')],
    [('HR', 'HR', 'R', 'HR', 'N'), ('HR', 'HR', 'R', 'N', 'HR')],
    [('HR', 'R', 'R', 'HR', 'HR'), ('HR', 'N', 'HR', 'R', 'N')],
    [('R', 'HR', 'R', 'N', 'HR'), ('HR', 'HR', 'N', 'HR', 'HR')],
    [('R', 'N', 'N', 'HR', 'R'), ('N', 'R', 'N', 'HR', 'N')],
    [('N', 'R', 'R', 'N', 'HR'), ('N', 'N', 'HR', 'HR', 'R')],
    [('R', 'N', 'N', 'HR', 'N'), ('R', 'HR', 'N', 'HR', 'N')],
    [('HR', 'N', 'N', 'R', 'HR'), ('HR', 'HR', 'N', 'R', 'HR')],
    [('HR', 'HR', 'R', 'N', 'N'), ('N', 'R', 'R', 'R', 'HR')],
    [('HR', 'R', 'HR', 'R', 'R'), ('N', 'N', 'R', 'R', 'N')],
    [('HR', 'HR', 'R', 'R', 'N'), ('HR', 'HR', 'HR', 'N', 'HR')],
    [('R', 'R', 'N', 'N', 'N'), ('R', 'HR', 'N'

    [('HR', 'N', 'R', 'HR', 'R'), ('HR', 'N', 'R', 'N', 'R')],
    [('HR', 'N', 'R', 'R', 'HR'), ('N', 'R', 'HR', 'R', 'N')],
    [('N', 'N', 'R', 'N', 'HR'), ('HR', 'R', 'HR', 'R', 'HR')],
    [('N', 'R', 'N', 'N', 'N'), ('N', 'R', 'HR', 'N', 'HR')],
    [('N', 'HR', 'N', 'R', 'HR'), ('N', 'HR', 'N', 'HR', 'R')],
    [('HR', 'R', 'R', 'N', 'HR'), ('R', 'R', 'HR', 'HR', 'HR')],
    [('R', 'R', 'HR', 'HR', 'N'), ('N', 'N', 'HR', 'HR', 'R')],
    [('R', 'N', 'HR', 'R', 'N'), ('HR', 'R', 'N', 'HR', 'R')],
    [('N', 'HR', 'R', 'HR', 'R'), ('N', 'R', 'N', 'HR', 'R')],
    [('N', 'R', 'N', 'R', 'R'), ('N', 'HR', 'R', 'HR', 'HR')],
    [('N', 'R', 'N', 'N', 'N'), ('R', 'HR', 'R', 'HR', 'R')],
    [('N', 'R', 'N', 'R', 'HR'), ('HR', 'R', 'R', 'R', 'R')],
    [('N', 'N', 'N', 'N', 'R'), ('N', 'N', 'N', 'R', 'N')],
    [('HR', 'N', 'N', 'HR', 'N'), ('N', 'N', 'R', 'N', 'HR')],
    [('R', 'R', 'N', 'HR', 'HR'), ('N', 'R', 'HR', 'HR', 'R')],
    [('R', 'N', 'N', 'HR', 'N'), ('HR', 'R', 'N', 'N', 

    [('HR', 'R', 'HR', 'N', 'HR'), ('HR', 'R', 'R', 'HR', 'N')],
    [('HR', 'HR', 'HR', 'R', 'R'), ('R', 'HR', 'N', 'N', 'HR')],
    [('R', 'N', 'N', 'N', 'HR'), ('R', 'R', 'HR', 'HR', 'N')],
    [('HR', 'HR', 'HR', 'R', 'HR'), ('R', 'HR', 'N', 'HR', 'R')],
    [('HR', 'N', 'HR', 'HR', 'N'), ('HR', 'R', 'N', 'R', 'HR')],
    [('HR', 'HR', 'HR', 'N', 'HR'), ('N', 'HR', 'N', 'HR', 'R')],
    [('HR', 'N', 'R', 'HR', 'HR'), ('N', 'R', 'R', 'HR', 'HR')],
    [('N', 'R', 'N', 'HR', 'N'), ('R', 'HR', 'N', 'R', 'HR')],
    [('HR', 'N', 'R', 'R', 'N'), ('HR', 'R', 'HR', 'HR', 'N')],
    [('HR', 'N', 'HR', 'R', 'HR'), ('R', 'HR', 'R', 'HR', 'HR')],
    [('N', 'N', 'R', 'N', 'N'), ('HR', 'R', 'HR', 'N', 'HR')],
    [('HR', 'N', 'N', 'HR', 'N'), ('HR', 'R', 'HR', 'HR', 'HR')],
    [('R', 'R', 'R', 'HR', 'R'), ('HR', 'R', 'N', 'R', 'R')],
    [('HR', 'N', 'N', 'N', 'HR'), ('R', 'HR', 'N', 'N', 'R')],
    [('HR', 'R', 'R', 'N', 'N'), ('HR', 'N', 'R', 'HR', 'HR')],
    [('N', 'R', 'HR', 'R', 'N'), (

Step 2

In [6]:
def precision_at_k(ranking_pair, k):
    precision = 0 
    if k > len(ranking_pair):
        k = len(ranking_pair)
    for i in range(k):
        if not ranking_pair[i] == 'N':
            precision += 1
            
    return precision / k

k = 10
precision_p = precision_at_k(pairs[1000][0], k) 
precision_e = precision_at_k(pairs[1000][1], k)
print(pairs[1000])
print(precision_p)
print(precision_e)
print(precision_e - precision_p)

[('N', 'N', 'N', 'R', 'R'), ('N', 'R', 'N', 'N', 'R')]
0.4
0.4
0.0


In [7]:
graded_relevance = {'N': 0, 'R': 1, 'HR': 2}

def nDCG_at_k(ranking_pair, k, highest_rel):
    nDCG = 0
    best_nDCG = 0
    if k > len(ranking_pair):
        k = len(ranking_pair)
    for i in range(1, k+1):
        rel = graded_relevance[ranking_pair[i-1]]
        nDCG += (2**rel - 1)/(math.log2(1 + i))
        
        # Assuming ideal is ["HR", "HR", "HR", "HR", "HR"]
        best_nDCG += (2**highest_rel - 1) / (math.log2(1 + i))
    return  nDCG / best_nDCG

nDCG_p = nDCG_at_k(pairs[1000][0], k, graded_relevance['HR'])
nDCG_e = nDCG_at_k(pairs[1000][1], k, graded_relevance['HR'])
print(pairs[1000])
print(nDCG_p)
print(nDCG_e)
print(nDCG_e - nDCG_p)

[('N', 'N', 'N', 'R', 'R'), ('N', 'R', 'N', 'N', 'R')]
0.09242447578501607
0.11506378074895644
0.02263930496394037


In [9]:
# Based on ERR paper
def relevance_probabilities(graded_relevance):
    rel_prob = {}
    max_value = max(list(graded_relevance.values()))
    for key, value in graded_relevance.items():
        rel_prob[key] = ((2**value) - 1) / (2**max_value)
    return rel_prob


def ERR(ranking_pair, rel_prob):
    sum_r = 0
    for r in range(1, len(ranking_pair) + 1):
        phi_r = rel_prob[ranking_pair[r-1]]
        if r != 1:
            prod_i = 1 
        else:
            prod_i = 0
        for i in range(1, r):
            phi_i = rel_prob[ranking_pair[i-1]]
            prod_i *= (1-phi_i) * phi_r
        sum_r += (1/r) * prod_i
    return sum_r
            

rel_prob = relevance_probabilities(graded_relevance)
print(pairs[1000])
ERR_p = ERR(pairs[1000][0], rel_prob)
print(ERR_p)
ERR_e = ERR(pairs[1000][1], rel_prob)
print(ERR_e)
print(ERR_e - ERR_p)


[('N', 'N', 'N', 'R', 'R'), ('N', 'R', 'N', 'N', 'R')]
0.0044921875
0.1255859375
0.12109375


Step 3

In [43]:
def balanced_interleaving(ranking1, ranking2):
    I = []
    k_a = 1
    k_b = 1
    # RandBit()
    priority = np.random.randint(0, 2)
    print("A first: " + str(priority)) 
    while(k_a <= len(ranking1) and k_b <= len(ranking2)):
        if((k_a < k_b) or ((k_a == k_b) and priority == 1)):
            if(ranking1[k_a - 1] not in I):
                I.append(ranking1[k_a - 1])
            k_a += 1
        else:
            if (ranking2[k_b - 1] not in I):
                I.append(ranking2[k_b - 1])
            k_b += 1     
    return I

# random doc ids
doc_pairs = [('doc1', 'doc2', 'doc3', 'doc4', 'doc5'), ('doc1', 'doc3', 'doc6', 'doc7', 'doc8')]
I = balanced_interleaving(doc_pairs[0], doc_pairs[1])
print(I)

#### TODO: click credit
clicked = random_click_model(2, 7)

A first: 0
['doc1', 'doc3', 'doc2', 'doc6', 'doc7', 'doc4', 'doc8']


Step 4

In [11]:
def random_click_model(number_of_clicks, shown_docs):
    p = number_of_clicks / shown_docs
    clicked = []
    for i in range(shown_docs):
        clicked.append(np.random.choice([0, 1], p=[1-0.2, 0.2]))
    return clicked

# 1 means document at index is clicked
clicked = random_click_model(2, 5)
print(pairs[100][0])
print(clicked)

('N', 'N', 'N', 'N', 'N')
[1, 1, 0, 0, 0]


In [2]:
def SDBN(sessions, urls_per_query):
    gamma = init_gamma(urls_per_query)
    alpha = init_alpha(len(sessions), urls_per_query)
    alpha = E_step(sessions, gamma, alpha, urls_per_query)
    print(alpha[0])
    gamma = M_step(sessions, gamma, alpha, urls_per_query)

def init_gamma(urls_per_query):
    return [0.1] * urls_per_query

def init_alpha(amount_of_sessions, urls_per_query):
    alpha = []
    for _ in range(amount_of_sessions):
        alpha.append([0.1] * urls_per_query)
    return alpha
    
def E_step(sessions, gamma, alpha, urls_per_query):
    url_sessions = retrieve_url_sessions(sessions)
    sum_alpha = [[0] * urls_per_query] * len(sessions)
    for url in url_sessions:
        for session_index in url_sessions[url]:
            session = sessions[session_index]
            urls = session[0][5:]
            rank = urls.index(url)
            clicked = 0
            if len(session) > 1:
                for click in session[1:]:
                    if click[3] == url:
                        clicked = 1
            sum_alpha[session_index][rank] += (1/len(url_sessions[url])) * clicked + ((1 - clicked) * ((1 - gamma[rank]) * alpha[session_index][rank]) / (1 - gamma[rank] * alpha[session_index][rank]))
    return sum_alpha
    
def retrieve_url_sessions(sessions):
    url_sessions = {}
    for i, session in enumerate(sessions):
        for url in session[0][5:]:
            if url in url_sessions:
                current_sessions = url_sessions[url]
                current_sessions.append(i)
                url_sessions[url] = current_sessions
            else:
                url_sessions[url] = [i]
    return url_sessions
            

def M_step(sessions, gamma, alpha, urls_per_query):
    sum_gamma = [0] * urls_per_query
    for i, session in enumerate(sessions):
        query = session[0]
        urls = query[5:]
        clicked_urls = []
        if len(session) > 1:
            for click in session[1:]:
                clicked_urls.append(click[3])
        for rank, url in enumerate(urls):
            alpha_u = alpha
            if url in clicked_urls:
                clicked = 1
            else:
                clicked = 0
            sum_gamma[rank] += clicked + ((1 - clicked) * ((1 - alpha[i][rank]) * gamma[rank]) / (1 - gamma[rank] * alpha[i][rank]))
    new_gamma = [old_gamma/len(sessions) for old_gamma in sum_gamma]
    return new_gamma

def determineProbability():
    return None

def documentClicked(p):
    return np.random.choice([0, 1], p=p)

# Store data per session
def read_click_log():
    with open('YandexRelPredChallenge.txt', 'r') as f:
        data = []
        i = -1
        while(i < 10000):
            row = f.readline().split()
            if row[2] == 'Q' or i == -1:
                i += 1
                data.append([])
                data[i].append(row)
            elif row[2] == 'C':
                data[i].append(row)
        return data

data = read_click_log()
SDBN(data, 10)




[2428.3534706784017, 1795.8082898463138, 1567.5202948507601, 1451.398009043361, 1333.7646243116064, 1265.065936375802, 1251.8471695903422, 1196.7616959997624, 1186.5070184980423, 1182.7937205974683]
